In [ ]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import re
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
import torchvision.transforms as transforms
from torchvision.io import read_image
from torch.utils.data import Dataset
from sklearn.model_selection import KFold
import random
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
!unzip drive/My\ Drive/NN_201_Sarmanova/full_data.zip #full_data.zip located in NN_201_Sarmanova folder at google drive.

Archive:  drive/My Drive/NN_201_Sarmanova/full_data.zip
  inflating: full_data/1.csv         
  inflating: full_data/10.csv        
  inflating: full_data/100.csv       
  inflating: full_data/1000.csv      
  inflating: full_data/101.csv       
  inflating: full_data/102.csv       
  inflating: full_data/103.csv       
  inflating: full_data/104.csv       
  inflating: full_data/105.csv       
  inflating: full_data/106.csv       
  inflating: full_data/107.csv       
  inflating: full_data/108.csv       
  inflating: full_data/109.csv       
  inflating: full_data/11.csv        
  inflating: full_data/110.csv       
  inflating: full_data/111.csv       
  inflating: full_data/112.csv       
  inflating: full_data/113.csv       
  inflating: full_data/114.csv       
  inflating: full_data/115.csv       
  inflating: full_data/116.csv       
  inflating: full_data/117.csv       
  inflating: full_data/118.csv       
  inflating: full_data/119.csv       
  inflating: full_data/12.csv   

#Customized data class

In [ ]:
# MLP_41W approach
class FCNN_41W_Dataset(Dataset):
    def __init__(self, annotations_file, spec_dir, transform=None, target_transform=None, sep = ","):
        self.spec_labels = pd.read_csv(annotations_file, sep=',').iloc[:,1:]
        self.spec_number = pd.read_csv(annotations_file, sep=',').iloc[:,0]
        self.spec_dir = spec_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.spec_labels)

    def __getitem__(self, idx):
        label = self.spec_labels.iloc[idx]
        
        sp = np.array(pd.read_csv(self.spec_dir + str(self.spec_number[idx])+'.csv').iloc[1:,1:-1], dtype='float32')
        sp[sp<0]=0
        spec = torch.from_numpy(sp)
        spec = (torch.reshape(spec, (41,500))).flatten()
        
        
        if self.transform:
            spec = self.transform(spec)
        if self.target_transform:
            label = self.target_transform(label)
        return spec, torch.from_numpy(np.array(label, dtype='float32'))

In [ ]:
# MLP_3W approach
class FCNN_3W_Dataset(Dataset):
    def __init__(self, annotations_file, spec_dir, transform=None, target_transform=None, sep = ","):
        self.spec_labels = pd.read_csv(annotations_file, sep=',').iloc[:,1:]
        self.spec_number = pd.read_csv(annotations_file, sep=',').iloc[:,0]
        self.spec_dir = spec_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.spec_labels)

    def __getitem__(self, idx):
        label = self.spec_labels.iloc[idx]
        
        sp = np.array(pd.read_csv(self.spec_dir + str(self.spec_number[idx])+'.csv').iloc[1:,[1,21,41]], dtype='float32')
        sp[sp<0]=0
        spec = torch.from_numpy(sp)
        spec = (torch.reshape(spec, (3,500))).flatten()
        
        if self.transform:
            spec = self.transform(spec)
        if self.target_transform:
            label = self.target_transform(label)
        return spec, torch.from_numpy(np.array(label, dtype='float32'))

In [ ]:
# MLP_1W approach
class FCNN_1W_Dataset(Dataset):
    def __init__(self, annotations_file, spec_dir, transform=None, target_transform=None, sep = ","):
        self.spec_labels = pd.read_csv(annotations_file, sep=',').iloc[:,1:]
        self.spec_number = pd.read_csv(annotations_file, sep=',').iloc[:,0]
        self.spec_dir = spec_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.spec_labels)

    def __getitem__(self, idx):
        label = self.spec_labels.iloc[idx]
        sp = np.array(pd.read_csv(self.spec_dir + str(self.spec_number[idx])+'.csv').iloc[1:,21], dtype='float32')
        sp[sp<0]=0
        spec = torch.from_numpy(sp)
        if self.transform:
            spec = self.transform(spec)
        if self.target_transform:
            label = self.target_transform(label)
        return spec, torch.from_numpy(np.array(label, dtype='float32'))

In [ ]:
class Normalize1D:
    def __init__(self,mean,std):
        assert mean.shape == std.shape
        self.mean = mean
        self.std = std

    def __call__(self,x):
        # x of shape [channel, value]
        #assert x.shape[0] == self.mean.shape[0]
        #assert len(x.shape) == 1
        mean = self.mean
        std = self.std
        return (x - std) / mean

In [ ]:
#Calculate mean and std
def mean_std(train_dataloader):
    data = next(iter(train_dataloader))
    mean = data[0].mean(dim=0)
    std = data[0].std(dim=0)

    return mean, std

#Basic Models

In [ ]:
# MLP_41W approach
class FCNN_41W(nn.Module):

    def __init__(self):
      
        super().__init__() # since Python 3.0

        self.layers = nn.Sequential(
            nn.Linear(20500,1000),
            nn.ReLU(),
            nn.Linear(1000, 4)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# MLP_3W approach
class FCNN_3W(nn.Module):

    def __init__(self):
      
        super().__init__() # since Python 3.0

        self.layers = nn.Sequential(
            nn.Linear(1500,64),
            nn.ReLU(),
            nn.Linear(64, 4)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
# MLP_1W approach
class FCNN_1W(nn.Module):

    def __init__(self):
      
        super().__init__() # since Python 3.0

        self.layers = nn.Sequential(
            nn.Linear(500,64),
            nn.ReLU(),
            nn.Linear(64, 4)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
def reset_weights(m):
    '''
    Try resetting model weights to avoid
    weight leakage.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
#Write the outputs of the network
def write_predictions(N, model_name, split_path,dataloader,dset):
    checkpoint = torch.load(split_path + 'model'+model_name+'.pth')
    N.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    last_best_epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    N.eval()

    y_ae = np.zeros((1,4))
    y_ae_true = np.zeros((1,4))
    for specs, labels in dataloader:
        outputs = N(specs)
        outputs[outputs<0]=0
        ae = outputs.detach().numpy()
        ae_true = labels.detach().numpy()
        #np.concatenate((y_ae, ae), axis=0)
        y_ae = np.concatenate((y_ae, ae), axis=0)
        y_ae_true = np.concatenate((y_ae_true, ae_true), axis=0)

    a = ['Cu','Ni','Cr','NO3']
    pd.DataFrame(y_ae).to_csv(split_path + 'Y_out_'+dset+'.csv',sep=',', header = a)
    pd.DataFrame(y_ae_true).to_csv(split_path + 'Y_true_'+dset+'.csv',sep=',', header = a)

In [ ]:
import math

def calculate_metrics(model, loader):
    running_mae = torch.zeros(1,4)
    running_mse = torch.zeros(1,4)
    y_sum = torch.zeros(1,4)
    num_examples = 0
    total_sum_squares = torch.zeros(1,4)
    target_mean = torch.zeros(1,4)

    for specs, targets in loader:
        y_sum += targets.sum(dim=0)
        num_examples += specs.shape[0]

    num_examples = 0
    for specs, targets in loader:
        preds = model(specs)
        preds[preds<0]=0

        num_examples += specs.shape[0] # batch size
        total_sum_squares += (torch.pow(preds - y_sum, 2)).sum(dim=0)

        error = torch.abs(preds - targets).sum(dim=0)
        squared_error = ((preds - targets)*(preds - targets)).sum(dim=0)
        running_mae += error
        running_mse += squared_error
  
  
    mae = (running_mae/num_examples).detach().numpy()
    rmse = (torch.sqrt(running_mse/num_examples)).detach().numpy()
    r2 = (1 - squared_error / total_sum_squares).detach().numpy()

    return mae, rmse, r2

# Prepare cross-validation datasets


In [ ]:
gen_path = 'full_data/'

case_name = 'FCNN_1W_100_0_001_baseline' # the name of the experiment
# case_name = model_name + stopping_criterion + learning_rate + comment

cnn_1d_path = case_name+'/'
Y = pd.read_csv(gen_path+'Y_ions.csv', sep=';')

k_folds = [[42,12],[612,45],[72,172],[871,48],[52,134]] #cross-validation folds

'''Write files with labels for each set (training/validation/test) within cross-validation fold - annotation files for CDS_FCNN_1W_Dataset'''

for fold in k_folds:
    split_path = cnn_1d_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '/'
    os.makedirs(split_path, exist_ok=True)

    Y_trn, Y_30 = train_test_split(Y, test_size=0.3, random_state=fold[0])
    Y_vld, Y_tst = train_test_split(Y_30, test_size = 0.3333, random_state=fold[1])

    a = ['sample_number','Cu','Ni','Cr','NO3']

    pd.DataFrame(Y_trn).to_csv(split_path + 'Y_trn.csv',sep=',', index=False, header = a)
    pd.DataFrame(Y_vld).to_csv(split_path + 'Y_vld.csv',sep=',', index=False, header = a)
    pd.DataFrame(Y_tst).to_csv(split_path + 'Y_tst.csv',sep=',', index=False, header = a)

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 8.8 MB/s 
     |████████████████████████████████| 180 kB 60.5 MB/s 
     |████████████████████████████████| 97 kB 5.7 MB/s 
     |████████████████████████████████| 140 kB 36.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#Training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = FCNN_1W()

loss_function = torch.nn.MSELoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)


for fold in k_folds:
    split_path = cnn_1d_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '/'
    
    
    training_data =  FCNN_1W_Dataset(split_path+'Y_trn.csv',gen_path)

    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
    mean, std = mean_std(train_dataloader)

    #Data import and normalization
    training_data =  FCNN_1W_Dataset(split_path+'Y_trn.csv', gen_path, transform= transforms.Compose([Normalize1D(mean,std)]))

    validation_data =  FCNN_1W_Dataset(split_path+'Y_vld.csv', gen_path, transform= transforms.Compose([Normalize1D(mean,std)]))
    test_data =  FCNN_1W_Dataset(split_path+'Y_tst.csv', gen_path, transform= transforms.Compose([Normalize1D(mean,std)]))

    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
    validation_dataloader = DataLoader(validation_data, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
    

    for init_number in range(0,3): #Это я делаю множественную инициализацию весов сети

        init_path = split_path + str(init_number)+'\\'
        os.makedirs(init_path, exist_ok=True)

        # Обучение
        # критерием остановки является ошибка на валидационном наборе - останавливаемся,
        #если в течение 100 эпох (test_stop) ошибка на валидационном наборе (val_loss) не падала

        test_stop = 100 #stopping criterion
        max_val_loss = 10000.0

        wandb.init(project = case_name)

        split_name = 'split_'+ str(fold[0])+'_'+str(fold[1])
        init_name = '_' + str(init_number)
        model_name = '_FCNN_1W'
        wandb.run.name = split_name + init_name + model_name
        wandb.run.save()

        for epoch_step in range(0, 1000, test_stop):
            
            if epoch_step!=0:
                checkpoint = torch.load(init_path + 'model'+model_name+'.pth')
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                last_best_epoch = checkpoint['epoch']
                loss = checkpoint['loss']
                model.train()
                
                if last_best_epoch + test_stop > ep:
                    for ep in range(epoch_step, epoch_step+test_stop):
                        for _, data in enumerate(train_dataloader, 0): # get bacth
                            inputs, labels = data # parse batch
                            optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                            outputs = model(inputs) # get outputs
                            loss = loss_function(outputs, labels) # calculate loss
                            loss.backward() # calculate gradients
                            optimizer.step() # performs a single optimization step (parameter update).

                        dl = 0
                        val_loss = 0.0
                        for specs, labels in validation_dataloader: 
                            val_loss += loss_function(model(specs),labels)
                            dl+=1
                        val_loss = val_loss/dl
                        
                        if val_loss.item() <= max_val_loss:
                            torch.save({'epoch': ep,
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'loss': loss}, init_path + 'model'+model_name+'.pth')
                            max_val_loss = val_loss.item()
                        wandb.log({"trn_loss": loss, "vld_loss": val_loss})
                else: continue    
      
            if epoch_step==0:
                model.apply(reset_weights)

                for ep in range(epoch_step, test_stop):
                    for _, data in enumerate(train_dataloader, 0): # get bacth
                        inputs, labels = data # parse batch
                        optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                        outputs = model(inputs) # get outputs
                        loss = loss_function(outputs, labels) # calculate loss
                        loss.backward() # calculate gradients
                        optimizer.step() # performs a single optimization step (parameter update).

                    dl = 0
                    val_loss = 0.0
                    for specs, labels in validation_dataloader:
                        val_loss += loss_function(model(specs),labels)
                        dl+=1
                    val_loss = val_loss/dl

                    if val_loss.item() <= max_val_loss:
                        torch.save({'epoch': ep,
                          'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'loss': loss}, init_path + 'model'+model_name+'.pth')
                        max_val_loss = val_loss.item()
                    wandb.log({"trn_loss": loss, "vld_loss": val_loss})

        trn_metrics = calculate_metrics(model, train_dataloader)
        vld_metrics = calculate_metrics(model, validation_dataloader)
        tst_metrics = calculate_metrics(model, test_dataloader)

        wandb.log({"trn_mae": trn_metrics[0].sum()/4, "trn_rmse": trn_metrics[1].sum()/4,"trn_r2": trn_metrics[2].sum()/4})
        wandb.log({"vld_mae": vld_metrics[0].sum()/4, "vld_rmse": vld_metrics[1].sum()/4,"vld_r2": vld_metrics[2].sum()/4})
        wandb.log({"tst_mae": tst_metrics[0].sum()/4, "tst_rmse": tst_metrics[1].sum()/4,"tst_r2": tst_metrics[2].sum()/4})
        wandb.log({"epoch": ep})

        wandb.finish()

        write_predictions(model, model_name, init_path, train_dataloader, dset = 'trn')
        write_predictions(model, model_name, init_path, validation_dataloader, dset ='vld')
        write_predictions(model, model_name, init_path, test_dataloader, dset ='tst')



Using device: cpu


wandb: Currently logged in as: oe_sarmanova (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Reset trainable parameters of layer = Linear(in_features=500, out_features=64, bias=True)
Reset trainable parameters of layer = Linear(in_features=64, out_features=4, bias=True)


epoch,▁
trn_loss,█▄▂▂▁▂
trn_mae,▁
trn_r2,▁
trn_rmse,▁
tst_mae,▁
tst_r2,▁
tst_rmse,▁
vld_loss,█▃▂▁▁▁
vld_mae,▁
vld_r2,▁


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Reset trainable parameters of layer = Linear(in_features=500, out_features=64, bias=True)
Reset trainable parameters of layer = Linear(in_features=64, out_features=4, bias=True)


KeyboardInterrupt: ignored